<a href="https://colab.research.google.com/github/orenbara/braude_cloudcomp/blob/master/hw3/chatbot_only/chat_bot_for_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install firebase

In [24]:
import json
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files, drive
import matplotlib.colors as mcolors
from firebase import firebase
import numpy as np
from IPython.display import YouTubeVideo

In [38]:

FBconn = firebase.FirebaseApplication('https://braude-cloud-chatbot-default-rtdb.europe-west1.firebasedatabase.app/',None)

create_db = True # @param {type:"boolean"}
comment = "!!!!!!Change create_db to True only if you have access to Anna's drive!!!!!!" # @param {type:"string"}

if create_db:
  from google.colab import drive
  drive.mount('/content/drive/')
  # Create and upload the empty JSON data to Firebase
  FBconn.delete('/', None)
  empty_data = {}
  file_path_empty_json = '/content/drive/MyDrive/Colab Notebooks/Cloud Compute Course Braude/chatbot/Empty.json'
  with open(file_path_empty_json, 'w') as f:
      json.dump(empty_data, f)
  with open(file_path_empty_json) as f:
      data = json.load(f)
  result_put = FBconn.put('/', 'empty_data', data)

  # Load JSON data
  file_path = '/content/drive/MyDrive/Colab Notebooks/Cloud Compute Course Braude/chatbot/Students.json'
  with open(file_path) as f:
      data = json.load(f)
  result_put = FBconn.put('/', 'OnShapeJsonData', data)
else:
  print("You didn't created the data base")


# Convert JSON data to a Data Frame
result_get = FBconn.get('/OnShapeJsonData/',None)
df = pd.DataFrame(result_get)




Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Chat bot

In [27]:
!pip install nltk

import nltk
from nltk.chat.util import Chat, reflections

In [43]:
import nltk
from nltk.chat.util import Chat, reflections
from firebase import firebase
from collections import Counter

# Initialize the connection to Firebase - DONE AT FB CREATION

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Function to fetch data from Firebase
def fetch_data_from_firebase():
    result = FBconn.get('/OnShapeJsonData/', None)
    if result:
        return result
    else:
        return {}

# Fetch data from Firebase
data = fetch_data_from_firebase()
df_data = pd.DataFrame(data)


# Process the data
users = Counter(df_data['User'].dropna())
documents = Counter(df_data['Document'].dropna())
tabs = Counter(df_data['Tab'].dropna())
descriptions = Counter(df_data['Description'].dropna())

###################################

# Custom pretty print function
def pretty_print_counter(counter, title):
    print(title)
    print("-" * len(title))
    for key, count in counter.items():
        print(f"{key}: {count}")
    print()

# Pretty print the counters
pretty_print_counter(users, "Users")
pretty_print_counter(documents, "Documents")
pretty_print_counter(tabs, "Tabs")
#pretty_print_counter(descriptions, "Descriptions")



###################################


# Calculate some statistics
total_actions = len(data)
unique_users = len(users)
unique_documents = len(documents)
unique_tabs = len(tabs)

# Define patterns and responses
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Welcome to the project management assistant.']),
    (r'how are you?', ['I\'m functioning well, thank you!', 'I\'m operational and ready to assist with your project management.']),
    (r'what is your name?', ['I\'m the Project Management Assistant.', 'You can call me the Assistant.']),
    (r'exit|bye|goodbye', ['Thank you for using the Project Management Assistant. Goodbye!', 'Farewell! Don\'t hesitate to return if you need more assistance with your project.']),
    (r'How many total actions are recorded?', [f"There are {total_actions} total actions recorded in the database."]),
    (r'Who is the most active user?', [f"The most active user is {users.most_common(1)[0][0]} with {users.most_common(1)[0][1]} actions."]),
    (r'How many unique users are there?', [f"There are {unique_users} unique users in the database."]),
    (r'What is the most frequently accessed document?', [f"The most frequently accessed document is '{documents.most_common(1)[0][0]}' with {documents.most_common(1)[0][1]} accesses."]),
    (r'How many different documents were accessed?', [f"There were {unique_documents} different documents accessed."]),
    (r'What is the most common tab used?', [f"The most commonly used tab is '{tabs.most_common(1)[0][0]}' with {tabs.most_common(1)[0][1]} uses."]),
    (r'What is the most frequent action description?', [f"The most frequent action description is '{descriptions.most_common(1)[0][0]}' occurring {descriptions.most_common(1)[0][1]} times."]),
    (r'What are the top 3 most active users?', [f"The top 3 most active users are: 1. {users.most_common(3)[0][0]} ({users.most_common(3)[0][1]} actions), 2. {users.most_common(3)[1][0]} ({users.most_common(3)[1][1]} actions), and 3. {users.most_common(3)[2][0]} ({users.most_common(3)[2][1]} actions)."]),
    (r'What is the least used tab?', [f"The least used tab is '{tabs.most_common()[-1][0]}' with only {tabs.most_common()[-1][1]} uses."]),
    #(r'How many different types of actions \(descriptions\) are there?', [f"There are {len(descriptions)} different types of actions (unique descriptions) in the database."]),
    (r'What actions did StudentA1 perform?', [f"StudentA1 performed actions like {descriptions.most_common(4)[2][0]}"]),
    #(r'Can you list all the documents accessed by {user}?', [f"{users} accessed the following documents: {documents}"]),
    (r'Which user accessed the {tab} tab the most?', [f"The user who accessed the {tabs} tab the most is {users.most_common(1)[0][0]} with {users.most_common(1)[0][1]} accesses."]),
    (r'Which tab was used the least by {user}?', [f"The least used tab by {users} is {tabs.most_common()[-1][0]} with only {tabs.most_common()[-1][1]} uses."])
]

# Create a chatbot
chatbot = Chat(patterns, reflections)

# Start the conversation
print("Hello! I'm the Project Management Assistant. How can I help you today?")
while True:
    user_input = input("You: ")

    if user_input.lower() in ['exit', 'bye', 'goodbye']:
        print("ChatBot: Thank you for using the Project Management Assistant. Goodbye!")
        break

    response = chatbot.respond(user_input)
    print("ChatBot:", response if response is not None else "This type of question is not supported.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Users
-----
StudentA: 503
StudentB: 799
StudentZ: 69
StudentC: 268

Documents
---------
1st lab: 1639

Tabs
----
N/A: 615
Part Studio 3: 152
Assembly 3: 153
Part Studio 1: 124
Assembly 1: 455
Assembly 3 Copy 1: 22
Document as a whole: 10
Assembly 2: 29
Part Studio 4: 38
Part Studio 2: 41

Hello! I'm the Project Management Assistant. How can I help you today?
You: How many total actions are recorded?
ChatBot: There are 1639 total actions recorded in the database.
You: Who is the most active user?
ChatBot: The most active user is StudentB with 799 actions.
You: How many unique users are there?
ChatBot: There are 4 unique users in the database.
You: What is the most frequently accessed document?
ChatBot: The most frequently accessed document is '1st lab' with 1639 accesses.
You: How many different documents were accessed?
ChatBot: There were 1 different documents accessed.
You: What is the most common tab used?
ChatBot: The most commonly used tab is 'N/A' with 615 uses.
You: What is the m

KeyboardInterrupt: Interrupted by user